In [689]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
import glob
from datetime import timedelta, datetime
from tqdm import tqdm, trange
import itertools
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import normalize
from tqdm import tqdm 
from sklearn import preprocessing

def mkdir(path):
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
    else:
        print(path+'目錄已存在')
        
def read(file):
    csv = r".csv"
    xlsx = r".xlsx"
    if file.endswith(csv):
        with open(file, encoding='unicode_escape') as q:
            output_df = pd.read_csv(q)
    elif file.endswith(xlsx):
        with open(file, 'rb') as q:
            output_df = pd.read_excel(q)
    return output_df
    
def feature_distribution_plot(non_padding_df):   
    """
    non_padding_df : DataFame type, should be in non-padding type
    output : .png type, in the folder 'feature_distribution'
    """
    filepath = "./feature_distribution_0127"
    mkdir(filepath)
    while os.path.exists(filepath):
        if ('serial' in non_padding_df.columns):
            non_padding_df = non_padding_df.drop(columns = ['serial'])
        if ('id' in non_padding_df.columns):
            non_padding_df = non_padding_df.drop(columns = ['id'])
        if ('date' in non_padding_df.columns):
            non_padding_df = non_padding_df.drop(columns = ['date'])
        non_padding_df = non_padding_df.dropna()
        
        for index in non_padding_df.columns:
            plt.figure(figsize=(8,6))
            sns.boxplot(x='value', y=index, data= non_padding_df)
            my_file = '0127_'+index+'.png'
            plt.savefig(my_file)
            
def append_id(life_df):
    """
    function : combine the 'PR_xxx' into lifestyle data
    life_df : DataFame type, should contain column named serial.
    output : DataFame type, contain column named id.
    """
    serial_id = []
    serial_life = life_df['serial'].tolist() #serial
    for i in range(0, len(serial_life)):
        con = serial_life[i]
        pr = str(map_df[map_df['id'] == con]['serial'].values).strip('[]').strip("''")
        serial_id.append(pr)
    id_df = pd.DataFrame(serial_id,columns=['id'])
    comp = pd.concat([life_df,id_df],axis=1)
    print('ID combination finish')
    comp_D = []
    comp_Date = comp['date'].tolist()
    for i in range(0, len(comp_Date)):
        con = comp_Date[i]
        pr = str(con).strip('00:00:00+08:00').strip('T')
        comp_D.append(pr)
    comp['date'] = pd.DataFrame(comp_D)
    return comp 

def append_serial(life_df):
    """
    function : combine the 'serial' into other data
    life_df : DataFame type, should contain column named serial.
    output : DataFame type, contain column named id.
    """
    serial_id = []
    serial_life = life_df['id'].tolist() #serial
    for i in range(0, len(serial_life)):
        con = serial_life[i]
        pr = str(map_df[map_df['serial'] == con]['id'].values).strip('[]').strip("''")
        serial_id.append(pr)
        print(con, pr)
    id_df = pd.DataFrame(serial_id,columns=['serial'])
    comp = pd.concat([life_df,id_df],axis=1)
    print('ID combination finish')
    return comp 

def map_ques(life_df, ques_df):
    """
    function : mapping questionnaire into lifestyle dataframe.
    life_df : DataFame type, should contain column named serial.
    output : DataFame type, contain column named id.
    """
    # deal with questionnaire
    sip = ques_df.copy()
    sip = sip.dropna(subset=['date'])
    ques_column = ['Age','id','Mmrc','1_move', '2_selfcare', '3_activity', '4_uncomfor', '5_depress','今天健康狀況','date',
        'CAT_1', 'CAT_2', 'CAT_3', 'CAT_4', 'CAT_5', 'CAT_6', 'CAT_7', 'CAT_8', 'CAT_total','AE 日期 (二週內)','anti-biotics',
       'steroid', 'bronchodilators']
    sip = sip.reindex(columns=ques_column)
#     #date of questionnaire format modifing
#     ques_D = []
#     ques_Date = ques_df['date'].tolist()
#     for i in range(0, len(ques_Date)):
#         con = ques_Date[i]
#         pr = str(con).strip(' 00:00:00')
#         ques_D.append(pr)
#     ques_df['date'] = pd.DataFrame(ques_D)

    sip['date'] = pd.to_datetime(sip['date'], errors='coerce')
    life_df['date'] = pd.to_datetime(life_df['date'], errors='coerce')
#     rrr = sip[sip['date'].isna()].index
#     print(rrr)
    comp = pd.merge(life_df, sip, on=['id','date'], how='left')
    res = comp.to_excel('./process_file/map_ques.xlsx',index=False)
    return comp

def check_missing_perc(df):
    '''
    function : to check empty percentage of the dataframe
    df : DataFrame type
    '''
    qwe=df.isnull().sum().sort_values(ascending=False)/len(df)
    asd=qwe[qwe>0]
    print(asd)
    return asd

def extract_hos_time(seg_df, lif_df):
    '''
    function : extract collecting patient time
    seg_df : DataFrame type, contain columns named 'id','serial','起','迄'
    lif_df : DataFrame type
    '''
    seg_df = seg_df.reindex(columns = ['id','serial','起','迄'])
    lif_df['date'] = pd.to_datetime(lif_df['date'])
    lif_df.index = lif_df['date']
    lif_range = []
    for ser in lif_df['serial'].unique():
        con = lif_df[lif_df['serial'] == ser]
        con = con.sort_index()
        if len(seg_df[seg_df['serial'] == ser]['起'].values) == 0:
            continue
        else:
            start_date = seg_df[seg_df['serial'] == ser]['起'].values[0]
            end_date = seg_df[seg_df['serial'] == ser]['迄'].values[0]
            date_range = con[start_date : end_date]
            lif_range.append(date_range)
    vom = pd.DataFrame()
    for ind in lif_range:
        vom = pd.concat([vom, ind], axis = 0)
    return vom

def merge_open_data(ope_df, lif_df):
    '''
    function : merge open data
    ope_df : DataFrame type, extracted open data
    lif_df : DataFrame type
    '''
    ope_df['date'] = pd.to_datetime(ope_df['date'])
    ope_df.index.name = None
    lif_df.index.name = None
    comp = pd.merge(lif_df, ope_df, on=['id','date','serial'], how='left')
    rest = comp.to_excel('./process_file/merge_open_data.xlsx', index=False)
    return comp

def listing_day(ae_day):
    time_sec = []
    for i in range(7):
        n = timedelta(days=i)
        x_day = ae_day - n
        time_sec.append(x_day)
    return time_sec

def labeling(ques_df, life_df):
    '''
    function : labeling ground truth
    ques_df : DataFrame type, contain column named 'AE 日期 (二週內)'
    life_df : DataFrame type
    '''
    columns = ['id','AE 日期 (二週內)']
    ae_date = ques_df[ques_df['AE 日期 (二週內)'].notnull()][columns]
    ae = []
    for id,content in ae_date.values :
        time_sec = listing_day(content)
        ae_data = ques_df[ques_df['AE 日期 (二週內)'] == content]
        uni_data = ae_data[ae_data['id'] == id]
        ae_whom = str(uni_data['id'].values).strip('[]').strip("''")
        for i in time_sec:  
            ae_when = str(i).strip('00:00:00').strip(' ')
            ae_value = True
            ae.append([ae_whom,ae_when,ae_value])
    ae = pd.DataFrame(ae, columns = ['id', 'date', 'value'])
    ae['date'] = pd.to_datetime(ae['date'])
    ae = ae.drop_duplicates()
    comp = pd.merge(life_df, ae, on=['id','date'], how = 'left')
    # input false
    pend_num = comp['value'].shape[0]
    value_queue = []
    for i in comp['value']:
        if i == True:
            value_queue.append(True)
        else:
            value_queue.append(False)
    comp['value'] = pd.DataFrame(value_queue)
#     res = comp.to_excel('./process_file/labeling.xlsx', index=False)
    return comp

def patient_choosing(patient_list):
    if patient_list == 'patient':
        patient_list = {'1':'7KL867','2':'k8toglom','3':'7MBVPK','4':'7WBRYJ','6':'kczxv1yv','7':'kavrxlv0',
                  '8':'kalrxuor','9':'7TNLYF','10':'7RWBXQ','11':'kadbivrk','12':'7RW6GF','13':'kbbnvugt','14':'7W3P7C',
                  '15':'kceij9pg','16':'k47r2n3q','17':'k2x2ub0g','18':'kdtwh1d2','19':'ke2r3z50','20':'k7odj3j9','21':'k47r9ign',
                  '22':'k8tmb0x8','23':'k3tipk3h','24':'7MB7NF','25':'6XJ5LS','26':'k5g3nwmg','27':'7MJ773','28':'kc04axbp',
                  '29':'k4nsgd6d','30':'kczuibsw','31':'7S2DJN','32':'kam6rhh8','33':'k9tmgc0q','34':'7L87WN','35':'karruqcp',
                  '36':'k7ohrrzh','37':'k8toglom','38':'k8nsxrf8','5':'k372eowt','126':'7J4LSJ',
                  '43':'k997cdna','44':'kam6ludx','45':'kcfkysl2','46':'7J6Y9D','47':'k9rss2np','48':'k3tj79oz','49':'kcpnwq06',
                  '50':'kaz56899','51':'k9j8fn98','52':'k40k27he','53':'k9rsmyeu','54':'k99aupqm','55':'ka3pehhb','56':'7M6XMQ',
                  '57':'7FZ7TY','58':'kaorfe00','59':'ka2aezge','60':'k4m6ts8z','61':'kbbm2myc','62':'7SMYHT','63':'7KMNHS',
                  '64':'kc04fomc','65':'k8xykf3o','66':'kb0dp1ue','67':'k99b5sby','68':'7RSN5M','69':'7SMD93','70':'k8nshpi3',
                  '71':'7MKC44','72':'7GYPY5','73':'kdr4e4wl','74':'7MKFFR','75':'7JZSZL','76':'ke0no4tg','77':'7GTFN9',
                  '78':'k7o6s8pl','79':'7HV84W','81':'7HMG67','82':'kdej4s8c','83':'7MHM4W','84':'7LXXQD',
                  '85':'7NFJKW','86':'783XWR','87':'7RMTVM','88':'k3dq74l6','89':'k93nsgzp','90':'2G7926','91':'kb1q0gqq',
                  '92':'7NQLKZ','93':'7LCWVV','94':'6WG8H8','95':'karou8wm','96':'7LHHQF','97':'7T3LYW','98':'kaerkis0',
                  '99':'karl5ctc','100':'k4m77n9j','101':'7L2KQW','102':'k3tiv86l','103':'k8nsub3m','104':'kc5xiieq','105':'7MB7XP',
                  '106':'kdtmxjjl','107':'k6iw0ywz','108':'k2tp0e2v','109':'k7o7u15e','110':'7RSHHF','111':'k4m7mszd','112':'7WGXYW',
                  '113':'kb1potm8','114':'kdi91iwz','115':'7JF7QJ','116':'7SDWSM','117':'7N93QH','119':'k2tpoc4a',
                  '120':'k3fnnnjn','121':'7Q8ZYG','122':'7MPFHS','123':'k42gwti3','124':'7V9Q5S','125':'7L87TF','127':'kcye41i8','128':'7KDJNB','129':'kc5vhfe0','130':'kb0gvtyo','131':'kdjilbd3',
                  '132':'k528rnky','133':'kdtw95p2','134':'kdeh9tt6','135':'kbbkpfoa','136':'kc5w4ob9','137':'7JWD4K','138':'ke2hfb32','139':'kceiba9o',
                       '139':'kgd66hpt','140':'khzm9k0j','141':'klvk4w7v','142':'km4vxg3p','143':'kn0dlkrs',
                       '144':'knfrg28p','145':'knfawjjo','146':'knfak07e','147':'kjtmvtd8','148':'kippk2c8','149':'kil0ffs0','150':'kia2b6g6',
                       '151':'ki5k4qzl','152':'ki3xbmyf','153':'khzsf37x','154':'khx6uk8f','155':'khpxb8hx','156':'khpx3qrg','157':'khjvrn4u',
                       '158':'kgq2lzjt','159':'kh604tvu','160':'kg8xc28s','161':'kg7pewto','162':'kfhtgs25','163':'kfdrhdqz','164':'kf54thjh','165':'kf3msaay',
                       '166':'kev6cplc','167':'kev689ab','168':'kerwnszh','169':'ke9qjtwg'}
    elif patient_list == 'NN_patient':
        patient_list = {'1':'karl5ctc','2':'karou8wm','3':'karruqcp','4':'kb0dp1ue','5':'kb1potm8','6':'kb1q0gqq','7':'kbbkpfoa',
                     '8':'kbbm2myc','9':'kbbnvugt','10':'kc5vhfe0','11':'kc5w4ob9','12':'kc5xiieq','13':'kcfkysl2','14':'kcpnwq06',
                     '15':'kczuibsw','16':'kczxv1yv','17':'kdej4s8c','18':'kdeh9tt6','19':'kdtw95p2','20':'kdtwh1d2','21':'ke2r3z50',
                       '22':'kgd66hpt','23':'khzm9k0j','24':'klvk4w7v','25':'km4vxg3p','26':'kn0dlkrs'}
    elif patient_list == 'NTU_patient':
        patient_list = {'1':'k773avug','2':'k7o7u15e','3':'k8nshpi3','4':'k7o6s8pl','5':'k7ohrrzh','6':'k8nsub3m','7':'k8nsxrf8',
                      '8':'k8tmb0x8','9':'k8toglom','10':'k8xykf3o','11':'k997cdna','12':'k99aupqm','13':'k99b5sby','14':'ka3pehhb',
                      '15':'k93nsgzp','16':'k9j8fn98','17':'k9rss2np','18':'k9rsmyeu','19':'kam6rhh8','20':'k9tmgc0q','21':'ka2aezge',
                      '22':'kalrxuor','23':'kals4gfo','24':'kadbivrk','25':'kaerkis0','26':'kam6ludx','27':'kaorfe00','28':'kavrxlv0',
                      '29':'kaz56899','30':'kb0gvtyo','31':'kc04axbp','32':'kc04fomc','34':'kceij9pg',
                      '36':'kcye41i8','37':'kdi91iwz','38':'ke0no4tg','39':'kdr4e4wl','40':'kdjilbd3','41':'7HMG67','42':'7HV84W',
                       '43':'7JWD4K','44':'7MBVPK','45':'7KMNHS','46':'7KL867','47':'7L87TF','48':'7MPFHS','49':'7MKFFR',
                      '50':'7MHM4W','51':'7MJ773','52':'7NFJKW','53':'7N93QH','54':'7RMTVM','55':'7RW6GF','56':'7RWBXQ',
                      '57':'7SDWSM','58':'7SMD93','59':'7V9Q5S','60':'7W3P7C','70':'7WGXYW','71':'2G7926','72':'k40k27he',
                      '73':'k47r2n3q','74':'k4m6ts8z','75':'k4nsgd6d','76':'knfrg28p','77':'knfawjjo','78':'knfak07e','79':'kjtmvtd8','80':'kippk2c8','81':'kil0ffs0','82':'kia2b6g6',
                       '83':'ki5k4qzl','84':'ki3xbmyf','85':'khzsf37x','86':'khx6uk8f','87':'khpxb8hx','88':'khpx3qrg','89':'khjvrn4u',
                       '90':'kgq2lzjt','91':'kh604tvu','92':'kg8xc28s','93':'kg7pewto','94':'kfhtgs25','95':'kfdrhdqz','96':'kf54thjh','97':'kf3msaay',
                       '98':'kev6cplc','99':'kev689ab','100':'kerwnszh','101':'ke9qjtwg'}
    elif patient_list == 'AE_attack_patient':
        patient_list = {'PR_004':'7HMG67','PR_005':'7HV84W','PR_010':'7JWD4K','PR_012':'7MBVPK','PR_014':'7KL867','PR_017':'7L87TF',
                        'PR_023':'7MPFHS','PR_025':'7MKC44','PR_026':'7MHM4W','PR_027':'7MJ773','PR_028':'7NFJKW','PR_029':'7N93QH',
                        'PR_034':'7RMTVM', 'PR_036':'7RW6GF','PR_037':'7RWBXQ','PR_038':'7SDWSM','PR_039':'7SMD93','PR_044':'7V9Q5S',
                        'PR_046':'7W3P7C','PR_048':'2G7926','PR_051':'k3fnnnjn','PR_056':'k40k27he','PR_061':'k4m6ts8z','PR_070':'k773avug',
                        'PR_074':'k7ohrrzh','PR_076':'k8nsxrf8','PR_079':'k8xykf3o','PR_080':'k997cdna','PR_082':'k99b5sby','PR_092':'kals4gfo',
                        'PR_095':'kam6ludx','PR_097':'kavrxlv0','PR_099':'kb0gvtyo','PR_102':'kceiba9o','PR_103':'kceij9pg','PR_104':'kcyki8mp','PR_105':'kcye41i8',
                        'PR_107':'ke0no4tg','PR_108':'kdr4e4wl', 'PR_109':'kdjilbd3', 'PR_116':'kev6cplc','PR_119':'kfdrhdqz', 'PR_121':'kg7pewto',
                        'PR_126':'khpx3qrg','PR_127':'khpxb8hx', 'PR_128':'khx6uk8f', 'PR_129':'khzsf37x','PR_135':'kjtmvtd8','PR_137':'knfawjjo',
                        'PR_N_002':'karou8wm', 'PR_N_004':'kb0dp1ue', 'PR_N_005':'kb1potm8', 'PR_N_008':'kbbm2myc', 'PR_N_012':'kc5xiieq','PR_N_014':'kcpnwq06',
                        'PR_N_015':'kczuibsw', 'PR_N_016':'kczxv1yv', 'PR_N_017':'kdej4s8c',  'PR_N_018':'kdeh9tt6','PR_N_019':'kdtw95p2','PR_N_020':'kdtwh1d2',
                        'PR_N_02':'ke2r3z50','PR_N_022':'kgd66hpt', 'PR_N_023':'khzm9k0j', 'PR_N_026':'klvk4w7v','PR_N_027':'km4vxg3p', 'PR_N_029':'kn0dlkrs'}
    return patient_list

def padding_ques(data_df, patient_list):
    df_columns = ['Mmrc','1_move', '2_selfcare', '3_activity', '4_uncomfor', '5_depress','今天健康狀況', 
                   'CAT_1', 'CAT_2', 'CAT_3', 'CAT_4', 'CAT_5', 'CAT_6', 'CAT_7', 'CAT_8', 'CAT_total','抗生素', '口服類固醇', '吸入短效支氣管擴張劑','value']
    life_columns = ['serial', 'date', 'avg_step', 'calories', 'q1_hr', 'q2_hr', 'q3_hr',  'temperature', 'humidity', 'pm25',
        'AQI', 'SO2SubIndex', 'COSubIndex', 'PM10SubIndex', 'NO2SubIndex', 'O38SubIndex', 'PM25SubIndex', 'wake',
       'rem', 'deep_sleep', 'light_sleep', 'id']
    patient_list = patient_choosing(patient_list)
    data_list= []
    for index,content in enumerate(patient_list):
        range_ser = data_df['serial']==patient_list[content]
        selected_df= data_df[data_df['serial']==patient_list[content]][df_columns]
        data_fill = selected_df.fillna(method='ffill')
        data_fmean = data_fill.fillna(data_fill.mean(axis=0))
        other_df = data_df[data_df['serial']==patient_list[content]][life_columns]
        sup = pd.concat([other_df, data_fmean], axis=1)
        data_list.append(sup)
    comp = pd.DataFrame()
    for item in (data_list):
        comp = pd.concat([comp,item],axis=0)
    data_df=comp
    data_df = data_df.drop_duplicates()
#     res = data_df.to_excel('./process_file/padding_ques.xlsx', index=False)
    
    return data_df

def padding(data_df, patient_list, method='MeanFilled', filename=' '):
    '''
    function : padding by different methods
    method : default as MeanFilled, 'MeanFilled', 'KNNImputer', or 'DropMiss'
    patient_list : default as all patient, 'patient', 'NN_patient', or 'NTU_patient'
    data_df : DataFrame type
    '''
    patient_list = patient_choosing(patient_list)
    data_list= []
    df_columns = ['Mmrc','1_move', '2_selfcare', '3_activity', '4_uncomfor', '5_depress','今天健康狀況', 
                   'CAT_1', 'CAT_2', 'CAT_3', 'CAT_4', 'CAT_5', 'CAT_6', 'CAT_7', 'CAT_8', 'CAT_total']
    life_columns = ['serial', 'date', 'avg_step', 'calories', 'q1_hr', 'q2_hr', 'q3_hr',
        'AQI', 'SO2SubIndex', 'COSubIndex', 'PM10SubIndex', 'NO2SubIndex', 'O38SubIndex', 'PM25SubIndex', 'wake',
       'rem', 'deep_sleep', 'light_sleep', 'id','value']
    if method == 'MeanFilled':
        for index,content in enumerate(patient_list):
            range_ser = data_df['serial']==patient_list[content]
            data_fill = data_df[range_ser].fillna(method='bfill')
            data_fmean = data_fill.fillna(data_fill.mean(axis=0))
            data_list.append(data_fmean)
        comp = pd.DataFrame()
        for item in (data_list):
            comp = pd.concat([comp,item],axis=0)
        data_df=comp
    elif method == 'KNNImputer':
        for index,content in enumerate(patient_list):
            selected_df= data_df[data_df['serial']==patient_list[content]][df_columns]
            other_df = data_df[data_df['serial']==patient_list[content]][life_columns]
            data_fill = selected_df.fillna(method='bfill')
            data_f = pd.concat([other_df,data_fill],axis=1)
            data_list.append(data_f)
        comp = pd.DataFrame()
        for item in (data_list):
            comp = pd.concat([comp,item],axis=0)
        data_df = comp
        info = data_df[['serial','date','id','value']]
        data_df2=data_df.drop(['serial','date','id','value'],axis=1) 
        imputer = KNNImputer()
        knn = imputer.fit_transform(data_df2)
        data_knn = pd.DataFrame(knn, columns = data_df2.columns)
        data_df = pd.concat([data_knn,info],axis=1)
    elif method == 'DropMiss':
        for index,content in enumerate(patient_list):
            range_ser = data_df['serial']==patient_list[content]
            selected_df= data_df[data_df['serial']==patient_list[content]][df_columns]
            data_fill = selected_df.fillna(method='bfill')
            data_life = data_df[range_ser][life_columns]
            data_patient = pd.concat([data_life,data_fill],axis=1)
            data_patient = data_patient.dropna()
            data_fmean = data_fill.fillna(data_fill.mean(axis=0))
            data_list.append(data_patient)
        comp = pd.DataFrame()
        for item in (data_list):
            comp = pd.concat([comp,item],axis=0)
        data_df=comp
    check_missing_perc(data_df)
    res = data_df.to_excel('./process_file/padding_'+str(filename)+'.xlsx',index=False)
    return data_df

def splitTrainTest(data_df, floder_name,test_size=0.3, file_name=' '):
    path = './Data-'+str(floder_name)
    mkdir(path)
    X_train, X_test= train_test_split(data_df, shuffle = True, test_size = 0.3, random_state=987)
    pw1 = X_train.to_excel(path+'/X_train_'+str(file_name)+'.xlsx', index=False)
    pw2 = X_test.to_excel(path+'/X_test_'+str(file_name)+'.xlsx', index=False)
    return X_train, X_test

def splitByDate(data_df, floder_name, train_size=0.6, valid_size=0.2, file_name=' '):
    path = './Data-'+str(floder_name)
    mkdir(path)
    data_df = data_df.sort_values(by='date')
    train_date_idx = int(len(data_df['date'].unique())*(train_size))
    valid_date_idx = int(len(data_df['date'].unique())*(train_size+valid_size))
    train_date = data_df['date'].unique()[train_date_idx]
    valid_date = data_df['date'].unique()[valid_date_idx]
    print(train_date, valid_date)
    train_df = data_df[data_df['date'] < train_date]
    valid_df = data_df[(train_date < data_df['date']) & (data_df['date'] < valid_date)]
    test_df = data_df[data_df['date'] > valid_date]
    pw1 = train_df.to_excel(path+'/train_df_'+str(file_name)+'.xlsx', index=False)
    pw2 = test_df.to_excel(path+'/test_df_'+str(file_name)+'.xlsx', index=False)
    pw2 = valid_df.to_excel(path+'/valid_df_'+str(file_name)+'.xlsx', index=False)
    return train_df, valid_df, test_df

def split(data_df, floder_name,valid_size=0.3, test_size=0.3, file_name=' '):
    path = './Data-'+str(floder_name)
    mkdir(path)
    X_train, X_test = train_test_split(data_df, shuffle = True, test_size = test_size, random_state=987)
    X_train, X_valid= train_test_split(X_train, shuffle = True, test_size = valid_size, random_state=987)
    pw1 = X_train.to_excel(path+'/X_train_'+str(file_name)+'.xlsx', index=False)
    pw2 = X_test.to_excel(path+'/X_test_'+str(file_name)+'.xlsx', index=False)
    pw3 = X_valid.to_excel(path+'/X_valid_'+str(file_name)+'.xlsx', index=False)
    return X_train, X_valid, X_test

def split_into_XY(data_df):
    Y = data_df['value']
    X = data_df.drop(columns=['serial', 'date','id', 'value'])
    return X,Y
def smote(X,Y):
    sm = SMOTE(sampling_strategy=0.6, random_state=999)
    X_smote, Y_smote = sm.fit_resample(X,Y)
    return X_smote, Y_smote
    
def split_smote(data_df, floder_name,test_size=0.3, valid_size=0.3,sampling='SMOTE', file_name=' '):
    '''
    function : split data into different size in random
    data_df : DataFrame type
    sampling : str type, 'SMOTE' or None
    file_name : str type
    '''
    path = './Data-'+str(floder_name)
    mkdir(path)
    #先丟掉AirBox
    data_df = data_df.dropna()
    if 'temperature' in data_df.columns:
        Y = data_df['value']
        columns = ['avg_step', 'calories', 'q1_hr', 'q2_hr', 'q3_hr', 
                   'AQI', 'SO2SubIndex', 'COSubIndex', 'PM10SubIndex', 'NO2SubIndex',
               'O38SubIndex', 'PM25SubIndex', 'wake', 'rem', 'deep_sleep',
               'light_sleep', 'Mmrc', '1_move', '2_selfcare',
               '3_activity', '4_uncomfor', '5_depress','今天健康狀況', 'CAT_1', 'CAT_2', 'CAT_3',
               'CAT_4', 'CAT_5', 'CAT_6', 'CAT_7', 'CAT_8', 'CAT_total',]
        X = data_df.reindex(columns = columns)
    #1.TF ratio fixed依照比列切出40/30/30
    # oversampling
    if sampling == 'SMOTE':
        sm = SMOTE(sampling_strategy=0.6, random_state=999)
        X_smote, Y_smote = sm.fit_resample(X,Y)
        X_ratrain_df, X_ravalid_df, Y_ratrain_df, Y_ravalid_df= train_test_split(X_smote, Y_smote,shuffle = True, test_size = valid_size, random_state=987)
        X_ratrain_df, X_ratest_df, Y_ratrain_df, Y_ratest_df= train_test_split(X_ratrain_df, Y_ratrain_df, shuffle = True, test_size = test_size, random_state=987)
        xl1 = X_ratrain_df.to_excel(path+'/X_train_1v1_'+str(file_name)+'.xlsx', index=False)
        yl1 = Y_ratrain_df.to_excel(path+'/Y_train_1v1_'+str(file_name)+'.xlsx', index=False)
        xl2 = X_ratest_df.to_excel(path+'/X_test_1v1_'+str(file_name)+'.xlsx', index=False)
        yl2 = Y_ratest_df.to_excel(path+'/Y_test_1v1_'+str(file_name)+'.xlsx', index=False)
        xl3 = X_ravalid_df.to_excel(path+'/X_valid_1v1_'+str(file_name)+'.xlsx', index=False)
        yl3 = Y_ravalid_df.to_excel(path+'/Y_valid_1v1_'+str(file_name)+'.xlsx', index=False)
#     print(X_ratrain_df.shape, X_ravalid_df.shape,X_ratest_df.shape, Y_ratrain_df.shape, Y_ravalid_df.shape, Y_ratest_df.shape)
#     print(Y_ratrain_df[Y_ratrain_df==True].shape,Y_ravalid_df[Y_ravalid_df==True].shape,Y_ratest_df[Y_ratest_df==True].shape)
    elif sampling == None:
    #2.隨機切出40/30/30
        X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, shuffle = True, test_size = 0.3, random_state=987)
        X_train, X_test, Y_train, Y_test= train_test_split(X_train, Y_train, shuffle = True, test_size = 0.3, random_state=987)
        pw1 = X_train.to_excel(path+'/X_train_'+str(file_name)+'.xlsx', index=False)
        py1 = Y_train.to_excel(path+'/Y_train_'+str(file_name)+'.xlsx', index=False)
        pw2 = X_test.to_excel(path+'/X_test_'+str(file_name)+'.xlsx', index=False)
        py2 = Y_test.to_excel(path+'/Y_test_'+str(file_name)+'.xlsx', index=False)
        pw3 = X_valid.to_excel(path+'/X_valid_'+str(file_name)+'.xlsx', index=False)
        py3 = Y_valid.to_excel(path+'/Y_valid_'+str(file_name)+'.xlsx', index=False)
#     print(X_train.shape, X_valid.shape,X_test.shape, Y_train.shape, Y_valid.shape, Y_test.shape)
#     print(Y_train[Y_train==True].shape,Y_valid[Y_valid==True].shape,Y_test[Y_test==True].shape)

def clip_outlier(data_df, cols):
    data_df[cols]= data_df[cols].clip(lower= data_df[cols].quantile(0.15), upper= data_df[cols].quantile(0.85), axis=1)
    return data_df

def clip_outlier_total(data_df):
    mp = data_df.reindex(columns=['avg_step', 'calories', 'q1_hr', 'q2_hr', 'q3_hr',
        'AQI', 'SO2SubIndex', 'COSubIndex',
       'PM10SubIndex', 'NO2SubIndex', 'O38SubIndex', 'PM25SubIndex', 'wake',
       'rem', 'deep_sleep', 'light_sleep', 'Mmrc', '1_move',
       '2_selfcare', '3_activity', '4_uncomfor', '5_depress', '今天健康狀況',
       'CAT_1', 'CAT_2', 'CAT_3', 'CAT_4', 'CAT_5', 'CAT_6', 'CAT_7', 'CAT_8',
       'CAT_total'])
    for cols in mp.columns:
        data_df[cols]= data_df[cols].clip(lower= data_df[cols].quantile(0.15), upper= data_df[cols].quantile(0.85))
    return data_df

def normalization(X):
    X_norm = normalize(X)
    X_norm = pd.DataFrame(X_norm)
    return X_norm

def label_CAT(ques_df, life_df):
    refans = life_df['value']
    cat_ram = ques_df.reindex(columns=['id', 'date','CAT_total'])
    cat_ram = cat_ram.dropna(subset=['CAT_total'])
    value = []
    for i in range(0, len(cat_ram['CAT_total'])):
        perInfo = cat_ram['id'].iloc[i]
        if i == 0:
            continue
        elif cat_ram['CAT_total'].iloc[i] + 2 >= cat_ram['CAT_total'].iloc[i-1]:
            now = cat_ram['date'].iloc[i]
            unc = listing_day(now)
            for j in unc:
                value.append([perInfo, j, True])
    value_df = pd.DataFrame(value, columns= ['id', 'date', 'replace_value'])
    comp = pd.merge(life_df, value_df, on=['id','date'], how = 'left')
    origValue = comp['value'].to_list()
    replValue = comp['replace_value'].to_list()
    newValue = []
    for i in range(0, len(origValue)-1):
        if type(replValue[i]) == float:
            newValue.append(origValue[i])
        else:
            newValue.append(replValue[i])
    cimp = pd.DataFrame(newValue, columns=['value'])
    nwl_df =life_df.reindex(columns=['serial', 'date', 'avg_step', 'calories', 'q1_hr', 'q2_hr', 'q3_hr',
       'temperature', 'humidity', 'pm25', 'wake', 'rem', 'deep_sleep',
       'light_sleep', 'id', 'Mmrc', '1_move', '2_selfcare', '3_activity',
       '4_uncomfor', '5_depress','今天健康狀況', 'CAT_1', 'CAT_2', 'CAT_3', 'CAT_4', 'CAT_5',
       'CAT_6', 'CAT_7', 'CAT_8', 'CAT_total', 'AE 日期 (二週內)', 'SiteId',
       'SiteName', 'AQI', 'SO2SubIndex', 'COSubIndex', 'PM10SubIndex',
       'NO2SubIndex', 'O38SubIndex', 'PM25SubIndex'])
    new_df = pd.concat([nwl_df,cimp], axis=1)
    return new_df



def differential(data_df):
    origin_columns = data_df.drop(['serial', 'date', 'id', 'Mmrc', '1_move', '2_selfcare', '3_activity',
                                   '4_uncomfor', '5_depress', 'CAT_1', 'CAT_2', 'CAT_3', 'CAT_4', 'CAT_5',
                                   'CAT_6', 'CAT_7', 'CAT_8', 'CAT_total', 'value'], axis=1)
    tmp_columns = ['diff_' + i for i in origin_columns.columns]
    tmp = pd.DataFrame([], columns=tmp_columns)
    for id in data_df['serial'].unique():
        tmp_data = data_df[data_df['serial'] == id]
        tmp_data[tmp_columns] = tmp_data[origin_columns.columns].diff().fillna(value=data_df.mean())
        tmp_data_modify = tmp_data[tmp_columns].drop_duplicates().T.drop_duplicates().T
        tmp = tmp.append(tmp_data_modify)
    data_df = pd.concat([data_df, tmp[tmp_columns]], axis=1)
    tmp_columns = ['diff2_' + i for i in origin_columns.columns]
    tmp = pd.DataFrame([], columns=tmp_columns)
    for id in data_df['serial'].unique():
        tmp_data = data_df[data_df['serial'] == id]
        tmp_data[tmp_columns] = tmp_data[origin_columns.columns].diff(periods=2).fillna(value=data_df.mean())
        tmp_data_modify = tmp_data[tmp_columns].drop_duplicates().T.drop_duplicates().T
        tmp = tmp.append(tmp_data_modify)
    data_df = pd.concat([data_df, tmp[tmp_columns]], axis=1)
    return data_df


def paddingAttenAE(data_df, patient_list, filename=' '):
    patient_list = patient_choosing(patient_list)
    data_list= []
    
    df_columns = ['Mmrc','1_move', '2_selfcare', '3_activity', '4_uncomfor', '5_depress','今天健康狀況', 
                   'CAT_1', 'CAT_2', 'CAT_3', 'CAT_4', 'CAT_5', 'CAT_6', 'CAT_7', 'CAT_8', 'CAT_total']
    life_columns = ['serial', 'date', 'avg_step',
                    'calories', 'q1_hr', 'q2_hr', 'q3_hr', 'AQI', 'SO2SubIndex', 'COSubIndex', 'PM10SubIndex',
                    'NO2SubIndex', 'O38SubIndex', 'PM25SubIndex', 'wake','rem', 'deep_sleep', 'light_sleep', 'id','value']
    for index,content in tqdm(enumerate(patient_list)):
        range_ser = data_df[data_df['serial']==patient_list[content]]
        for row in range_ser[range_ser['value']==True].itertuples():
            upper_row_index = data_df.iloc[row[0]-1][data_df.iloc[row[0]-1].isnull() == True].index
            downner_index = data_df.iloc[row[0]][data_df.iloc[row[0]].isnull() == True].index

            if (upper_row_index.all() == downner_index.all()):
                if  (upper_row_index.all() != []) & (downner_index.all() != []):
                    ae_row = pd.DataFrame(data_df.iloc[row[0]].fillna(range_ser[range_ser['value']==True].mean(axis=0)))
            else:
                ae_row = pd.DataFrame(data_df.iloc[row[0]].fillna(data_df.iloc[row[0]-1]))
            at_tow = ae_row.T
            data_list.append(at_tow)

        false_ser = range_ser[range_ser['value']==False]
        data_fill = false_ser.fillna(method='ffill')
        data_fmean = data_fill.fillna(data_fill.mean(axis=0))
        data_list.append(data_fmean)
    comp = pd.DataFrame()
    for item in (data_list):
        comp = pd.concat([comp,item],axis=0)
    data_df=comp
    res = data_df.to_excel('./process_file/paddingAttenAE_'+str(filename)+'.xlsx',index=False)
    return data_df
def padding_medicine(ques_df, life_df):
    df_columns = ['Age','anti-biotics',
       'steroid', 'bronchodilators']
    life_columns = ['serial', 'date', 'avg_step', 'calories', 'q1_hr', 'q2_hr', 'q3_hr',  'temperature', 'humidity', 'pm25',
        'AQI', 'SO2SubIndex', 'COSubIndex', 'PM10SubIndex', 'NO2SubIndex', 'O38SubIndex', 'PM25SubIndex', 'wake',
       'rem', 'deep_sleep', 'light_sleep', 'id']
    
    columns = ['id','AE 日期 (二週內)']
    age_columns = ['id','Age']
    comp = []
    for u in df_columns:
        if u == "Age":
            life_df = pd.merge(life_df, ques_df[age_columns], on=['id'], how='left')
        else:
            bio_date = ques_df[ques_df[u].notnull()][columns]
            ae = []
            for id,content in bio_date.values:
                time_sec = listing_day(content)
                bio_data = ques_df[ques_df['AE 日期 (二週內)'] == content]
                uni_data = bio_data[bio_data['id'] == id]
                ae_whom = str(uni_data['id'].values).strip('[]').strip("''")
                for i in time_sec:  
                    ae_when = str(i).strip('00:00:00').strip(' ')
                    bio_value = True
                    ae.append([ae_whom,ae_when,bio_value])
            ae = pd.DataFrame(ae, columns = ['id', 'date', u])
            ae['date'] = pd.to_datetime(ae['date'])
            ae = ae.drop_duplicates()
            comp = pd.merge(life_df, ae, on=['id','date'], how = 'left')
            # input false
            pend_num = comp[u].shape[0]
            value_queue = []
            for i in comp[u]:
                if i == True:
                    value_queue.append(True)
                else:
                    value_queue.append(False)
            comp[u] = pd.DataFrame(value_queue)
            life_df = pd.concat([life_df,comp[u]], axis=1)
#     res = comp.to_excel('./process_file/labeling.xlsx', index=False)
    return life_df

## Questionnaire concatenation
- adjust the header for questionnaires

In [738]:
# 總院問卷
ques_file = 'C:/Users/smiley/Desktop/COPD/總院redcap資料_2021_04_19.xlsx' 
ques_df = read(ques_file)
comp1 = ques_df['id'].fillna(method='ffill')
ques_df['id'] = comp1
comp1 = ques_df['主要診斷'].fillna(method='ffill')
ques_df['主要診斷'] = comp1
comp1 = ques_df['Age'].fillna(method='ffill')
ques_df['Age'] = comp1
comp2 = ques_df.reindex(columns = [ 'id','主要診斷','Age','Mmrc', '1_move', '2_selfcare',
       '3_activity', '4_uncomfor', '5_depress', '今天健康狀況', '　EQ-5D total score',
       'date', 'CAT_1', 'CAT_2', 'CAT_3', 'CAT_4', 'CAT_5', 'CAT_6', 'CAT_7',
       'CAT_8', 'CAT_total', 'AE 日期 (二週內)','anti-biotics',
       'steroid', 'bronchodilators'])
comp3 = comp2.dropna(subset=['date'])
ques_df = comp3
ques_df = ques_df[ques_df['主要診斷']=="COPD"]
ranL = []
for i in idD['id']:
    a = ques_df[ques_df['id']==i]
    ranL.append(a)
comp = pd.DataFrame()
for item in ranL:
    comp = pd.concat([comp, item], axis=0)
ques_df.to_excel('C:/Users/smiley/Desktop/COPD/20210614總院問卷.xlsx')
ques_df=comp

In [739]:
# 北護問卷
ques_file = 'C:/Users/smiley/Desktop/COPD/台大北護工作列表_20210409.xlsx' 
qubh_df = read(ques_file)
comp1 = qubh_df['id'].fillna(method='ffill')
qubh_df['id'] = comp1
comp1 = qubh_df['主要診斷'].fillna(method='ffill')
qubh_df['主要診斷'] = comp1
comp1 = qubh_df['Age'].fillna(method='ffill')
qubh_df['Age'] = comp1

comp2 = qubh_df.reindex(columns = [ 'id','主要診斷','Age','Mmrc', '1_move', '2_selfcare',
       '3_activity', '4_uncomfor', '5_depress', '今天健康狀況', '　EQ-5D total score',
       'date', 'CAT_1', 'CAT_2', 'CAT_3', 'CAT_4', 'CAT_5', 'CAT_6', 'CAT_7',
       'CAT_8', 'CAT_total', 'AE 日期 (二週內)','anti-biotics',
       'steroid', 'bronchodilators'])
comp3 = comp2.dropna(subset=['date'])
qubh_df = comp3
qubh_df = qubh_df[qubh_df['主要診斷']=="COPD"]
ranL = []
for i in idD['id']:
    a = qubh_df[qubh_df['id']==i]
    ranL.append(a)
comp = pd.DataFrame()
for item in ranL:
    comp = pd.concat([comp, item], axis=0)
qubh_df.to_excel('C:/Users/smiley/Desktop/COPD/20210614北護問卷.xlsx')
qubh_df=comp

In [740]:
ques_df = pd.concat([ques_df,qubh_df], axis=0)
ques_df.to_excel('C:/Users/smiley/Desktop/COPD/20210615ques.xlsx', index=False)

## Request the files

In [695]:
map_file = 'C:/Users/smiley/Desktop/COPD/Questionaire/0409_mapping.xlsx'
seg_file = 'C:/Users/smiley/Desktop/COPD/OrigData-address/withSiteName_0413_obser.xlsx' #含北護的
open_file = 'C:/Users/smiley/Desktop/COPD/YuOpenData_0420.csv' #含北護的
hour_file = 'C:/Users/smiley/Desktop/COPD/process_file/hour_to_avg_0519.xlsx' 
ques_file = 'C:/Users/smiley/Desktop/COPD/20210615ques.xlsx'
# hourbh_file = 'C:/Users/smiley/Desktop/COPD/process_file/hour_to_avg_bh.xlsx' 

map_df = read(map_file)
seg_df = read(seg_file)
open_df = read(open_file)
hour_df = read(hour_file)
ques_df = read(ques_file)

In [741]:
id_df = append_id(hour_df)
id_df.to_excel('./process_file/0616_id.xlsx')

ID combination finish


In [731]:
id_df = read('./process_file/0610_id.xlsx')
mp_df = map_ques(id_df, ques_df)
mp_df.to_excel('./process_file/0616_map.xlsx')

In [732]:
time_df = extract_hos_time(seg_df, mp_df)
time_df.to_excel('./process_file/0616time_df.xlsx')

In [733]:
openhs_df = merge_open_data(open_df, time_df)
openhs_df = openhs_df.drop_duplicates()
openhs_df.to_excel('./process_file/0616openhs_df.xlsx')

In [734]:
label_df = labeling(ques_df, openhs_df)
label_df.to_excel('./process_file/0616label_df.xlsx')

## check the shape of ground truth on first labeling

In [735]:
label_df[label_df['value']==True].shape

(256, 48)

In [736]:
cat_df = label_CAT(ques_df, label_df)
cat_df.to_excel('./process_file/0616cat_df.xlsx')

## check the shape of ground truth on second labeling

In [737]:
cat_df[cat_df['value']==True].shape

(728, 42)

## Dealing with missing values
- drop_all_data
- padding_all_data
    - filename: String

In [742]:
def drop_all_data(data_df):
    data_df= data_df.dropna()
    data_df = data_df.reset_index(drop=True)
    return data_df

In [742]:
comp_df = drop_all_data(cat_df)

In [743]:
def padding_all_data(data_df, filename):
    pad_df = paddingAttenAE(data_df, 'patient', filename=filename)
    return pad_df

In [ ]:
comp_df = padding_all_data(cat_df, filename)

## Data Normalization

In [745]:
def nornmalization(data_df):
    from sklearn import preprocessing
    scaler = data_df.reindex(columns=['avg_step', 'calories', 'q1_hr', 'q2_hr', 'q3_hr',
           'AQI', 'SO2SubIndex', 'COSubIndex', 'PM10SubIndex', 'NO2SubIndex',
           'PM25SubIndex', 'wake', 'rem', 'deep_sleep', 'light_sleep'])
    infodf = data_df.reindex(columns=['serial', 'date', 'id','value','anti-biotics', 'steroid', 'bronchodilators'])

    Min_Max_Scaler = preprocessing.MinMaxScaler(feature_range=(0,1)) # 設定縮放的區間上下限
    MinMax_Data = Min_Max_Scaler.fit_transform(scaler)
    
    nor_df = pd.DataFrame(MinMax_Data, columns=['avg_step', 'calories', 'q1_hr', 'q2_hr', 'q3_hr',
       'AQI', 'SO2SubIndex', 'COSubIndex', 'PM10SubIndex', 'NO2SubIndex',
       'PM25SubIndex', 'wake', 'rem', 'deep_sleep', 'light_sleep'])
    data_df = pd.concat([nor_df,infodf], axis =1)
    return data_df

In [ ]:
comp_df = nornmalization(comp_df)

In [724]:
comp_df=comp_df.reindex(columns=['serial', 'date', 'Age', 
       'avg_step', 'calories', 'q1_hr', 'q2_hr', 'q3_hr', 'AQI', 'SO2SubIndex',
       'COSubIndex', 'PM10SubIndex', 'NO2SubIndex', 'PM25SubIndex', 'wake',
       'rem', 'deep_sleep', 'light_sleep', 'id', 'value'])

## Data split
1. split by ratio

In [ ]:
train_df, valid_df, test_df = split(comp_df, 'Folder_name',test_size=0.1, valid_size=0.1, file_name='ntu')

2. split by date

In [726]:
train_df, valid_df, test_df = splitByDate(comp_df, 'Folder_name', train_size=0.6, test_size=0.2, file_name='ntu')

In [727]:
Xtrain, Ytrain = split_into_XY(train_df)
Ytrain=Ytrain.astype('int')
XsmoteTrain, YsmoteTrain = smote(Xtrain,Ytrain)

Xvalid, Yvalid = split_into_XY(valid_df)
Yvalid=Yvalid.astype('int')
XsmoteValid, YsmoteValid = smote(Xvalid,Yvalid)

Xtest, Ytest = split_into_XY(test_df)

- Xtrain & Ytrain: original training set
- Xvalid & Yvalid: original validating set
- XsmoteTrain & YsmoteTrain: oversampling training set
- XsmoteValid & YsmoteValid: oversampling validating set
- Xtest & Ytest: original testing set

In [728]:
Xtrain.to_excel('./Folder_name/Xtrain.xlsx', index=False)
Ytrain.to_excel('./Folder_name/Ytrain.xlsx', index=False)
Xvalid.to_excel('./Folder_name/Xvalid.xlsx', index=False)
Yvalid.to_excel('./Folder_name/Yvalid.xlsx', index=False)
XsmoteTrain.to_excel('./Folder_name/XsmoteTrain.xlsx', index=False)
YsmoteTrain.to_excel('./Folder_name/YsmoteTrain.xlsx', index=False)
XsmoteValid.to_excel('./Folder_name/XsmoteTest.xlsx', index=False)
YsmoteValid.to_excel('./Folder_name/YsmoteTest.xlsx', index=False)
Xtest.to_excel('./Folder_name/Xtest.xlsx', index=False)
Ytest.to_excel('./Folder_name/Ytest.xlsx', index=False)